In [87]:
import json
import codecs

def parseData(fname):
    for l in open(fname, 'rt', encoding="utf8"):
        yield eval(l)

file_name = ("reviews.json")
with codecs.open(file_name, 'r', encoding='utf-8',
                 errors='ignore') as f:
    data_t = list(parseData(file_name))[:60000]

In [22]:
data=[]
for i in data_t:
    if 'stars' in i.keys():
        data.append(i)

In [61]:
data[0]

{'work': '3206242',
 'flags': [],
 'unixtime': 1194393600,
 'stars': 5.0,
 'nhelpful': 0,
 'time': 'Nov 7, 2007',
 'comment': 'This a great book for young readers to be introduced to the world of Middle Earth. ',
 'user': 'van_stef'}

In [79]:
import numpy as np
import urllib
import nltk
from nltk.stem.porter import *
from sklearn import linear_model

import string
import math
import random
import re
import time
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


In [29]:
bigramCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in data:
    r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
    words = r.split()
    for i in range(0,len(words)-1):
        bigram = words[i] + " " + words[i+1]
        bigramCount[bigram] += 1

In [30]:
countsBigram = [(bigramCount[d], d) for d in bigramCount.keys()]
countsBigram.sort()
countsBigram.reverse()

In [31]:
bigrams = [c[1] for c in countsBigram[:1000]]
bigramId = dict(zip(bigrams, range(len(bigrams))))
bigramSet = set(bigrams)

In [32]:
def feature(text):
    feat = [0]*len(bigrams)
    words = text.split()
    for i in range(len(words)-1):
        bigram = words[i] + " " + words[i+1]
        try:
            feat[bigramId[bigram]] += 1
        except KeyError:
            continue
    feat.append(1) #offset
    return feat

In [33]:
reviewText = [''.join([c for c in datum['comment'].lower() if not c in punctuation]) for datum in data]
            

In [34]:
X_2 = []
for i in range(len(data)):
    X_2.append(feature(reviewText[i]))
y_2 = [d['stars'] for d in data]

In [37]:
reg = 1.0
clf = linear_model.Ridge(reg, fit_intercept=False)
clf.fit(X_2, y_2)
theta = clf.coef_
predictions = clf.predict(X_2)

In [ ]:
##############

In [40]:
unigramCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
    for w in r.split():
        unigramCount[w] += 1

In [41]:
countsUnigram = [(unigramCount[w], w) for w in unigramCount]
countsUnigram.sort()
countsUnigram.reverse()

In [42]:
unigrams = [x[1] for x in countsUnigram[:1000]]
unigramId = dict(zip(unigrams, range(len(unigrams))))
unigramSet = set(unigrams)

countsCombined = countsUnigram + countsBigram
countsCombined.sort()
countsCombined.reverse()

combineds = [x[1] for x in countsCombined[:1000]]
combinedId = dict(zip(combineds, range(len(combineds))))



In [43]:
def feature_3(text):
    feat = [0]*len(combineds)
    words = text.split()
    for i in range(len(words)-1):
        bigram = words[i] + " " + words[i+1]
        try:
            feat[combinedId[bigram]] += 1
        except KeyError:
            continue
    for w in words:
        try:
            feat[combinedId[w]] += 1
        except KeyError:
            continue
    feat.append(1) #offset
    return feat

In [45]:
X_3 = []
for i in range(len(data)):
    X_3.append(feature_3(reviewText[i]))
y_3 = [d['stars'] for d in data]

In [46]:
reg = 1.0
clf = linear_model.Ridge(reg, fit_intercept=False)
clf.fit(X_3, y_3)
theta = clf.coef_
predictions_3 = clf.predict(X_3)

In [47]:
mean_squared_error(y_3, predictions_3)
                  

0.7250240354521877

In [49]:
minIdx = np.argpartition(theta, 5)[0:5]
smallest = [(combineds[i], theta[i] ) for i in minIdx]
print ("Most negative associated weight:")
for x in smallest:
    print ("Unigram/Bigram: {:10s} \t with value: {:0.5f}".format(x[0], x[1]))



Most negative associated weight:
Unigram/Bigram: but this   	 with value: -0.31890
Unigram/Bigram: couldnt    	 with value: -0.27200
Unigram/Bigram: looking for 	 with value: -0.27189
Unigram/Bigram: the rest   	 with value: -0.24637
Unigram/Bigram: 3          	 with value: -0.24443


In [51]:
df = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
    words = set(r.split())
    for w in words:
        df[w] += 1

In [52]:
def idf(word):
    f = df[word]
    if f == 0:
      # Return maximum idf
        return log(len(data), 10)
    return log(len(data) / float(f))


def tf(word, reviewText):
    words = reviewText.split()
    c=0
    for w in words: 
        if w == word:
            c += 1
    return c
def tf_idf(word, reviewText):
    return tf(word, reviewText) * idf(word)


In [67]:
def feature_6(reviewText):
    feat = [0]*len(unigrams)
    words = reviewText.split()
    for w in words:
        try:
            feat[unigramId[w]] = tf_idf(w, reviewText)
        except KeyError:
            continue
    feat.append(1) #offset
    return feat

In [69]:
X_6 = np.array([feature_6(d) for d in reviewText])
y_6 = np.array([d['stars'] for d in data])

In [85]:
reg = 10.0
clf = linear_model.Ridge(reg, fit_intercept=True)
clf.fit(X_6, y_6)
theta = clf.coef_
predictions_6 = clf.predict(X_6)

In [89]:
np.sqrt(mean_squared_error(predictions_6, y_6))

0.8517558972984948